<a href="https://colab.research.google.com/github/trashchenkov/gigachat_tutorials/blob/main/%D0%90%D0%B3%D0%B5%D0%BD%D1%82%D1%8B_%D0%B2_%D0%93%D0%B8%D0%B3%D0%B0%D1%87%D0%B5%D0%B9%D0%BD%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Что такое агенты?

В этом туториале мы научимся создавать агентов. **Агенты** - это важная тема, постоянно сопровождающая дискуссии о больших языковых моделях.

Основное предназначение языковых моделей - это генерация текста на основе промпта. Но что будет, если языковую модель снабдить инструментами и инструкциями, как ими пользоваться? Сможет ли она эффективно решать реальные задачи?

Агент в этом смысле представляет собой языковую модель, которой предоставили **инструменты** и ставят задачи, а она *самостоятельно* выбирает пути решения и обращается к инструментам.

Под инструментами мы будем понимать питоновские функции. Языковая модель сможет их вызывать, передавая нужные аргументы и используя получаемые результаты. Таким образом, например, можно организовать доступ большой языковой модели в Интернет, чтобы она могла получить контекст для ответа на вопрос, выходящий за пределы доступных ей знаний. Или посчитать математический пример с помощью калькулятора, чтобы не допустить ошибок в вычислениях.

Для создания агентов нужны модели специальной разновидности, которые способны выдавать генерируемый ответ в определенном формате.

## Какого агента мы создадим?
В этом туториале мы создадим ассистента с набором инструментов. В качестве инструментов будут следующие функции:
- просмотра перечня заметок;
- создания новой заметки;
- просмотра содержимого отдельной заметки;
- поиска информации в Интернете;
- поиск роликов на ютубе и их саммаризация.

Для реализации такого агента нам понадобится ряд библиотек:
- gigachain для создания агента;
- duckduckgo-search для поиска в Сети;
- gradio для графического интерфейса диалога;
- youtube-search для поиска видео по запросу;
- youtube-search-python для извлечения метаданных о ролике;
- youtube-transcript-api для извлечения субтитров.



In [1]:
from google.colab import userdata
auth = userdata.get('SBER_AUTH')
auth1 = userdata.get('SBER1')

In [ ]:
!pip install -q gigachain
!pip install -q duckduckgo-search
!pip install -q youtube-transcript-api
!pip install -q youtube-search
!pip install -q gradio
!pip install -q youtube-search-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 843.3/843.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.5/292.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

## Предварительный пример

Прежде чем создавать сложного агента с большим количеством инструментов, давайте сделаем простой пример, чтобы посмотреть, как ведет себя модель при необходимости вызвать функцию.

In [2]:
from langchain.chat_models.gigachat import GigaChat

В создании объекта класса GigaChat нет ничего необычного, главное указать модель, которая точно поддерживает вызов функций. Для этого смотрите [документацию](https://developers.sber.ru/docs/ru/gigachat/models).

In [17]:
giga = GigaChat(credentials=auth1,
                model='GigaChat-Pro-preview',
                # пока название модели такое,
                # обещают, что скоро можно будет просто GigaChat указывать
                verify_ssl_certs=False,
                scope='GIGACHAT_API_CORP'
                )

In [3]:
giga = GigaChat(credentials=auth,
                model='GigaChat-preview',
                # пока название модели такое,
                # обещают, что скоро можно будет просто GigaChat указывать
                verify_ssl_certs=False
                )

Инструменты - это питоновские функции. Главное указать декоратор @tool и дать детальное описание того, что делает инструмент и как он это делает. На это описание будет опираться модель при выборе и вызове инструмента.

In [4]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Складывает числа a и b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Умножает a на b."""
    return a * b


tools = [add, multiply]

Далее список инструментов связываем с языковой моделью.

In [5]:
llm_with_tools = giga.bind_tools(tools)

При передаче запроса модели она может выбрать вызов функции-инструмента, если посчитает это необходимым.

In [7]:
query = "Сколько будет 3 * 12? А еще сколько будет 47 плюс 20"
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}, response_metadata={'token_usage': Usage(prompt_tokens=127, completion_tokens=17, total_tokens=144), 'model_name': 'GigaChat-preview:3.1.25.3', 'finish_reason': 'function_call'}, id='run-5836b97f-7264-4793-b969-8df65b787455-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '755e0085-4a3c-42c6-8062-f2dc2e89a193'}])

В примере выше модель сгенерировала ответ в виде запроса на вызов функции. Как видите, атрибут `content` вернулся пустым (обычно там сгенерированный текст). Зато в ответе есть такое:

```
additional_kwargs={'function_call': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}
```

Таким образом модель выдала нам структурированный ответ, который можно использовать в программном коде. В нашем случае мы четко видим название функции и передаваемые значения аргументов.

## Создание полноценного агента
Теперь, когда мы понимаем, как отвечает модель, чтобы вызвать инструмент, приступим к созданию полноценного ИИ-ассистента.

Для начала создадим все инструменты. Будем прописывать каждый инструмент в отдельной ячейке со своими импортами, чтобы было понятнее.

Условно, все инструменты из этого туториала можно разделить на две группы:
- работа с заметками;
- поиск информации в Интернете и на Ютубе.

Отдельным инструментом, который не вошел в эти две группы является функция по стиранию диалога (перезапуск беседы).

Сперва разберемся с инструментами для заметок.


### Работа с заметками

Несколько подготовительных действий:
- создадим папку под хранение заметок;
- для работы с файлами применим компонент `FileManagementToolkit`.





In [5]:
!mkdir Заметки
from langchain_community.agent_toolkits import FileManagementToolkit
file_tools = FileManagementToolkit(
    root_dir="/content/Заметки")

In [8]:
file_tools.get_tools()[-1].invoke({}).split('\n')

['No files found in directory .']

#### Инструмент для получения списка сохраненных заметок

In [6]:
from langchain_core.tools import tool

@tool
def get_list_of_notes() -> str:
  """Возвращает список файлов в папке заметок. В названии заметок \
  содержится тема заметки и ее порядковый номер.
  """
  check_notes = file_tools.get_tools()[-1]
  return check_notes.invoke({})

#### Инструмент для создания заметки

In [9]:
@tool
def write_note(topic: str, text: str) -> str:
  '''Записывает текст в заметку. Нужно передать тему в качестве части названия \
  будущего файла заметки (дата и время прибавлятся в название автоматически) и \
  текст самой заметки. В качестве ответа функция возвращает сообщение об успешной записи.
  '''
  writer_tool = file_tools.get_tools()[-2]
  check_notes = file_tools.get_tools()[-1]
  n = len(check_notes.invoke({}).split('\n'))
  return writer_tool.invoke({"file_path": topic+f' {n}'+'.txt', 'text': text})

#### Инструмент для чтения заметки

In [10]:
@tool
def read_note(file_name: str) -> str:
  '''Возвращает содержимое заметки в ответ на заданное имя файла. \
  Если имя указано неправильно, вернет сообщение об ошибке. \
  Имя файла заметки имеет вид "Тема заметки N.txt".
  '''
  read_tool = file_tools.get_tools()[-3]
  try:
    return read_tool.invoke({"file_path": file_name})
  except Exception as e:
    return f'Не удалось прочитать файл: {e}'

### Работа с внешними источниками

В качестве внешних источников информации выступает поисковик DuckDuckGo и YouTube.

#### Инструмент для поиска через DuckDuckGo

In [11]:
from langchain_community.tools import DuckDuckGoSearchResults

search =  DuckDuckGoSearchResults()

@tool
def search_web(query: str) -> str:
    """Поиск информации в Интернете. Передает запрос, возвращает выдачу поисковика.\
    Полученный результат надо обобщить и передать пользователю."""
    result = search.run(query)
    writer_tool = file_tools.get_tools()[-2]
    check_notes = file_tools.get_tools()[-1]
    n = len(check_notes.invoke({}).split('\n'))
    writer_tool.invoke({"file_path": f'Запрос: {query}'+f' {n}'+'.txt', 'text': result})
    return result

#### Инструмент для поиска и саммаризации видеороликов

In [14]:
from langchain_community.tools import YouTubeSearchTool
from youtube_transcript_api import YouTubeTranscriptApi
from youtubesearchpython import Video, ResultMode
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document
from datetime import datetime, timedelta
import json

video_template = '''
Название ролика: {}

Канал: {}

Продолжительность ролика: {}

Дата публикации: {}

Описание видео: {}


'''

yt = YouTubeSearchTool()
sum_chain = load_summarize_chain(giga, chain_type="map_reduce")


@tool
def find_video(query: str) -> str:
  '''Поиск видео на YouTube. Возвращает название видео, продолжительность, \
  название канала и краткое содержание (если получится вытащить субтитры)
  '''
  links = yt.run(query)
  links = json.loads(links.replace('\'', '"'))
  video_result = ''
  for i in links:
    video_id = i.split('=')[1]
    video_id = video_id.split('&')[0]
    try:
      yt_data = YouTubeTranscriptApi.get_transcript(video_id, languages=['ru', 'en'])
      text1 = []
      for j in yt_data:
        text1.append(j['text'])
        script = ' '.join(text1)
      doc = Document(page_content=script)
      split_docs = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_documents(
      [doc])
      res = sum_chain.run(split_docs)
    except Exception as e:
      res = 'Не удалось извлечь информацию.'
    video_info = Video.getInfo(video_id, mode = ResultMode.json)
    title = video_info["title"]
    channel = video_info["channel"]["name"]
    # Конвертация продолжительности видео из секунд в часы:минуты:секунды
    duration_seconds = int(video_info['duration']['secondsText'])
    duration_str = str(timedelta(seconds=duration_seconds))

    # Конвертация даты публикации в формат дд.мм.гг
    publish_date = datetime.strptime(video_info['publishDate'], "%Y-%m-%dT%H:%M:%S%z")
    publish_date_str = publish_date.strftime("%d.%m.%y")
    video_result += video_template.format(title, channel, duration_str, publish_date_str, res)
  writer_tool = file_tools.get_tools()[-2]
  check_notes = file_tools.get_tools()[-1]
  n = len(check_notes.invoke({}).split('\n'))
  writer_tool.invoke({"file_path": f'Видео: {query}'+f' {n}'+'.txt', 'text': video_result})
  return video_result


#### Инструмент для стирания истории сообщений в диалоге


In [12]:
@tool
def memory_clearing() -> None:
  '''Очищает историю диалога, позволяя начать разговор \
  пользователя и ассистента с чистого листа.
  '''
  global  chat_history_m
  chat_history_m = []

### Создаем непосредственно агента
Теперь, когда есть инструменты, можно создавать самого агента.

Тут нужно быть внимательным при написании кода, поскольку в примерах кода агентов [LangChain](https://python.langchain.com/docs/use_cases/tool_use/quickstart/) и [GigaChain](https://github.com/ai-forever/gigachain/blob/dev/docs/docs/modules/agents/how_to/gigachat_phone_agent.ipynb) есть разночтения. Опираемся на GigaChain, чтобы реализовать работу с GigaChat.

In [18]:
from langchain.agents import AgentExecutor, create_gigachat_functions_agent

tools = [get_list_of_notes, write_note, read_note, find_video, search_web, memory_clearing]
agent = create_gigachat_functions_agent(giga, tools)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
chat_history_m = []

## Делаем графический интерфейс в gradio
Создаем интерфейс и организуем чат-структуру сообщений.



In [19]:
import gradio as gr
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

def respond(message, chat_history):
    result = agent_executor.invoke(
        {
            "chat_history": chat_history_m,
            "input": message,
        }
    )
    chat_history_m.append(HumanMessage(content=message))
    chat_history_m.append(AIMessage(content=result["output"]))
    chat_history.append((message, result["output"]))
    return "", chat_history

def main():
    with gr.Blocks() as demo:
        with gr.Column():
            chatbot = gr.Chatbot()
            msg = gr.Textbox()
            msg.submit(respond, [msg, chatbot], [msg, chatbot])
    demo.launch(debug=True)

if __name__ == "__main__":
    main()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://28342f9688f801c763.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Здравствуйте! Я могу найти для вас информацию в интернете, написать текст или код на Python, сочинить песню, стихотворение или даже рассказ, а также поговорить с вами обо всём, что вас интересует.

> Finished chain.


> Entering new AgentExecutor chain...
У меня есть различные встроенные функции, такие как создание текстов, программирование на Python, ответы на общие вопросы, общение с вами и многое другое. Могу я чем-то помочь вам?

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_list_of_notes` with `{}`


Запрос: Сергей Тращенков 1.txt
Видео: Сергей Тращенков 1.txt[]

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `search_web` with `{'query': 'gigachain'}`
responded: Добавил в очередь на генерацию...

[snippet: pip install gigachain or pip install langsmith && conda install gigachain -c conda-forge. 🤔 What is this? Large language models (LLMs) are emerging as a transformative technology, enabling 